In [106]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [107]:
# Sample data
data = [
    {"context": "TensorFlow is an open-source platform for machine learning.", "question": "What is TensorFlow?", "answer": "an open-source platform for machine learning"},
    {"context": "Python is a programming language.", "question": "What is Python?", "answer": "a programming language"},
]

In [108]:

# Prepare dataset
contexts = [item["context"] for item in data]
questions = [item["question"] for item in data]
answers = [item["answer"] for item in data]


In [109]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(contexts + questions + answers)

context_sequences = tokenizer.texts_to_sequences(contexts)
question_sequences = tokenizer.texts_to_sequences(questions)

In [110]:

# Pad sequences
max_length = max(max(len(seq) for seq in context_sequences), max(len(seq) for seq in question_sequences))
context_padded = pad_sequences(context_sequences, maxlen=max_length)
question_padded = pad_sequences(question_sequences, maxlen=max_length)

In [111]:
# Prepare answer labels as start and end indices
start_labels = []
end_labels = []

In [112]:
for i, answer in enumerate(answers):
    context = contexts[i]
    start_index = context.find(answer)
    end_index = start_index + len(answer) - 1

    # Convert to token indices
    start_token_index = tokenizer.texts_to_sequences([context[:start_index]])[0][-1]
    end_token_index = tokenizer.texts_to_sequences([context[:end_index + 1]])[0][-1]

    start_labels.append(start_token_index)
    end_labels.append(end_token_index)

start_labels = np.array(start_labels)
end_labels = np.array(end_labels)

In [113]:
# Model Definition
input_context = layers.Input(shape=(max_length,))
input_question = layers.Input(shape=(max_length,))

embedding_layer = layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)

context_embedding = embedding_layer(input_context)
question_embedding = embedding_layer(input_question)

In [114]:
# Merge context and question
merged = layers.concatenate([context_embedding, question_embedding])
x = layers.LSTM(128)(merged)
x = layers.Dense(64, activation='relu')(x)

In [115]:
# Output layers for start and end positions
start_output = layers.Dense(len(tokenizer.word_index) + 1, activation='softmax', name='start')(x)
end_output = layers.Dense(len(tokenizer.word_index) + 1, activation='softmax', name='end')(x)

model = models.Model(inputs=[input_context, input_question], outputs=[start_output, end_output])

In [116]:
# Compile the model with metrics for both outputs
model.compile(optimizer='adam', 
              loss={'start': 'sparse_categorical_crossentropy', 'end': 'sparse_categorical_crossentropy'},
              metrics={'start': 'accuracy', 'end': 'accuracy'})

In [117]:

# Train the model
model.fit([context_padded, question_padded], [start_labels, end_labels], epochs=100, batch_size=2)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - end_accuracy: 0.0000e+00 - loss: 5.4277 - start_accuracy: 0.0000e+00
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - end_accuracy: 0.0000e+00 - loss: 5.3652 - start_accuracy: 1.0000
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - end_accuracy: 0.0000e+00 - loss: 5.3134 - start_accuracy: 1.0000
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - end_accuracy: 0.0000e+00 - loss: 5.2567 - start_accuracy: 1.0000
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - end_accuracy: 0.0000e+00 - loss: 5.1854 - start_accuracy: 1.0000
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - end_accuracy: 0.0000e+00 - loss: 5.0942 - start_accuracy: 1.0000
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - end_accuracy: 0.0000e+00 - loss: 4.9774 - start_accuracy: 1.0000
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - end_accuracy: 0.0000e+00 - loss: 4.8280 - start_accuracy: 1.0000
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - end_accura

In [118]:
model.save('owngpt.h5')

In [119]:
# Function to answer questions
def answer_question(context, question):
    context_seq = tokenizer.texts_to_sequences([context])
    question_seq = tokenizer.texts_to_sequences([question])
    
    context_padded = pad_sequences(context_seq, maxlen=max_length)
    question_padded = pad_sequences(question_seq, maxlen=max_length)

    start_pred, end_pred = model.predict([context_padded, question_padded])
    
    start_index = np.argmax(start_pred, axis=1)[0]
    end_index = np.argmax(end_pred, axis=1)[0]
     # Get answer from context based on predicted indices
    answer_tokens = context_padded[0][start_index:end_index + 1]
    answer = tokenizer.sequences_to_texts([answer_tokens])[0]

    return answer.strip()

In [120]:

# Example usage
context = "TensorFlow is an open-source platform for machine learning."
question = "What is TensorFlow?"

answer = answer_question(context, question)
print("Answer:", answer)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
Answer: is an open source platform for machine learning


In [121]:
# Example usage
context = "TensorFlow is a programming language."
question = "What is TensorFlow?"

answer = answer_question(context, question)
print("Answer:", answer)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Answer: tensorflow is a programming language
